In [1]:
# Standard libraries
import numpy as np
import pandas as pd
import warnings
import zipfile, io

# Visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

# Statistical libraries
import scipy.stats as ss
from scipy import stats
from scipy.stats import f_oneway
from scipy.sparse import csr_matrix

# Scikit-learn preprocessing and model selection
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler, OrdinalEncoder, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# Scikit-learn feature selection
from sklearn.feature_selection import f_classif, SelectKBest, mutual_info_classif, RFE, RFECV

# Scikit-learn models
from sklearn.linear_model import LogisticRegression, LassoCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

# Scikit-learn metrics
from sklearn.metrics import make_scorer, precision_score, recall_score, f1_score, classification_report

# Pickle for import and export of datasets
import pickle

# Seaborn settings
sns.set()

# Ignore warnings
warnings.filterwarnings('ignore')

# Enable inline plotting for Jupyter notebooks
%matplotlib inline

# 3. Data Scaling

* Due to the different types of information that Categorical features and Numerical features provide, each requires unique data scaling methods. Therefore, before normalizing features, we must split them into numeric and categorical sets.
* Regarding the numerical variables, the objective is to place them into the same scale, from 0 to 1, where 1 refers to the maximum value of the feature while 0 refers to the minimum value of the feature. One reason for normalizing / scaling the data is to prevent the model from giving more importance to variables with larger absolute numbers, since we do not want to make any assumptions on the level of importance of each feature. It is relevant to mention that while scaling we only consider information obtained from the training data. Only after we can use the minimum and maximum values of the training dataset for each variable to scale the validation dataset.
* Concerning categorical features, these are encoded to be readable by the models, as these do not typically process non-numeric data directly. To do so, we used the Ordinal Encoder method to transform the categorical variables by assigning each category an unique integer based on its order or occurrence


In [2]:
# load the data (train and test)
X_train = pd.read_csv('datasets/feature_engineering_train_delivery1.csv', index_col=0)
X_test = pd.read_csv('datasets/feature_engineering_test_delivery1.csv', index_col=0)

In [3]:
# Split the data into training features and target variable
y_train = X_train[['Claim Injury Type']]
X_train = X_train.drop('Claim Injury Type', axis=1)

In [4]:
X_train.columns

Index(['Age at Injury', 'Alternative Dispute Resolution',
       'Attorney/Representative', 'Average Weekly Wage', 'Birth Year',
       'C-2 Date', 'C-3 Date', 'Carrier Name', 'Carrier Type',
       'County of Injury', 'COVID-19 Indicator', 'District Name', 'Gender',
       'Industry Code', 'Industry Code Description', 'Medical Fee Region',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code',
       'Number of Dependents', 'Accident Year', 'Accident Month',
       'Accident on Weekday', 'Assembly Year', 'Assembly Month', 'Age Group',
       'Frequent Injury Cause', 'Broad Body Part'],
      dtype='object')

In [5]:
numeric_features = ['Age at Injury', 'Average Weekly Wage', 'Birth Year','Number of Dependents', 'Accident Year', 'Accident Month', 'Assembly Year','Assembly Month']
                    
categorical_features = ['Carrier Name', 'Carrier Type', 'County of Injury','District Name', 'Industry Code', 'Industry Code Description', 'Medical Fee Region',
                        'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
                        'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description', 'Age Group',
                        'WCIO Part Of Body Code', 'WCIO Part Of Body Description', 'Zip Code', 'Broad Body Part']
                        
binary_features = ['Alternative Dispute Resolution', 'Attorney/Representative','C-2 Date', 'C-3 Date', 'COVID-19 Indicator','Gender', 'Accident on Weekday','Frequent Injury Cause']

In [6]:
X_train_numeric = X_train[numeric_features]
X_train_categorical = X_train[categorical_features]
X_train_binary = X_train[binary_features]

X_test_numeric = X_test[numeric_features]
X_test_categorical = X_test[categorical_features]
X_test_binary = X_test[binary_features]

## 3.1 Data Scaling for Numerical

In [7]:
# Call function
scaler = MinMaxScaler()

# Fit to training data
scaler.fit(X_train_numeric)

# Transform training data
X_train_numeric_scaled = scaler.transform(X_train_numeric)

# Check results
X_train_numeric_scaled

array([[0.32692308, 0.        , 0.67346939, ..., 1.        , 0.        ,
        0.        ],
       [0.61538462, 0.89948867, 0.36734694, ..., 0.63636364, 0.        ,
        0.        ],
       [0.5       , 0.73919707, 0.48979592, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.36538462, 0.        , 0.69387755, ..., 1.        , 1.        ,
        1.        ],
       [0.90384615, 0.51059621, 0.12244898, ..., 1.        , 1.        ,
        1.        ],
       [0.19230769, 0.        , 0.87755102, ..., 1.        , 1.        ,
        1.        ]])

In [8]:
# Convert array into a pandas df

X_train_numeric_scaled = pd.DataFrame(X_train_numeric_scaled, columns=X_train_numeric.columns, index=X_train_numeric.index)
X_train_numeric_scaled.head()

,Age at Injury,Average Weekly Wage,Birth Year,Number of Dependents,Accident Year,Accident Month,Assembly Year,Assembly Month
Claim Identifier,,,,,,,,
5393875,0.326923,0.000000,0.673469,0.166667,0.930233,1.000000,0.0,0.0
5393091,0.615385,0.899489,0.367347,0.666667,0.930233,0.636364,0.0,0.0
5393889,0.500000,0.739197,0.489796,1.000000,0.930233,1.000000,0.0,0.0
5393887,0.903846,0.253005,0.061224,0.166667,0.930233,1.000000,0.0,0.0
5393848,0.653846,0.000000,0.326531,0.166667,0.930233,1.000000,0.0,0.0


In [9]:
# Scale test set too and convert it to pandas df

X_test_numeric_scaled = scaler.transform(X_test_numeric)
X_test_numeric_scaled = pd.DataFrame(X_test_numeric_scaled, columns=X_test_numeric.columns, index=X_test_numeric.index)

## 3.1 Data Scaling for Categorical

In [10]:
X_train_categorical.columns

Index(['Carrier Name', 'Carrier Type', 'County of Injury', 'District Name',
       'Industry Code', 'Industry Code Description', 'Medical Fee Region',
       'WCIO Cause of Injury Code', 'WCIO Cause of Injury Description',
       'WCIO Nature of Injury Code', 'WCIO Nature of Injury Description',
       'Age Group', 'WCIO Part Of Body Code', 'WCIO Part Of Body Description',
       'Zip Code', 'Broad Body Part'],
      dtype='object')

In [11]:
X_train_categorical.head()

,Carrier Name,Carrier Type,County of Injury,District Name,Industry Code,Industry Code Description,Medical Fee Region,WCIO Cause of Injury Code,WCIO Cause of Injury Description,WCIO Nature of Injury Code,WCIO Nature of Injury Description,Age Group,WCIO Part Of Body Code,WCIO Part Of Body Description,Zip Code,Broad Body Part
Claim Identifier,,,,,,,,,,,,,,,,
5393875,NEW HAMPSHIRE INSURANCE CO,1A. PRIVATE,ST. LAWRENCE,SYRACUSE,44.0,RETAIL TRADE,I,27.0,FROM LIQUID OR GREASE SPILLS,10.0,CONTUSION,Adult,62.0,BUTTOCKS,13662,Other
5393091,ZURICH AMERICAN INSURANCE CO,1A. PRIVATE,WYOMING,ROCHESTER,23.0,CONSTRUCTION,I,97.0,REPETITIVE MOTION,49.0,SPRAIN OR TEAR,Adult,38.0,SHOULDER(S),14569,Other
5393889,INDEMNITY INSURANCE CO OF,1A. PRIVATE,ORANGE,ALBANY,56.0,ADMINISTRATIVE AND SUPPORT AND WASTE MANAGEMEN...,II,79.0,OBJECT BEING LIFTED OR HANDLED,7.0,CONCUSSION,Adult,10.0,MULTIPLE HEAD INJURY,12589,Upper Limbs
5393887,STATE INSURANCE FUND,2A. SIF,DUTCHESS,ALBANY,62.0,HEALTH CARE AND SOCIAL ASSISTANCE,II,16.0,"HAND TOOL, UTENSIL; NOT POWERED",43.0,PUNCTURE,Old,36.0,FINGER(S),12603,Other
5393848,LM INSURANCE CORP,1A. PRIVATE,ONONDAGA,SYRACUSE,72.0,ACCOMMODATION AND FOOD SERVICES,I,19.0,"CUT, PUNCTURE, SCRAPE, NOC",40.0,LACERATION,Adult,36.0,FINGER(S),13029,Other


In [12]:
#We're having a cardinality issue, with a lot of features. One Hot Encoder isn't being able to run properly.
# We will assess that problem here and then maybe drop some redundant features.

# Check cardinality of categorical features
cardinality = X_train_categorical.nunique()
cardinality

Carrier Name                         1965
Carrier Type                            8
County of Injury                       63
District Name                           8
Industry Code                          24
Industry Code Description              20
Medical Fee Region                      5
WCIO Cause of Injury Code              77
WCIO Cause of Injury Description       74
WCIO Nature of Injury Code             56
WCIO Nature of Injury Description      56
Age Group                               4
WCIO Part Of Body Code                 57
WCIO Part Of Body Description          54
Zip Code                             6648
Broad Body Part                         6
dtype: int64

We need to fix the high cardinality issue before encoding the categorical features. We will use the following strategies:
  - Analyse the cardinality of each feature and check if we can drop it
    - Feature 1 with high cardinality : Carrier Name:
      - As we've seen in the EDA, the Carrier Name feature has a high correlation with Carrier Type. Given That, we are dropping it
    - Feature 2 with high cardinality: Zip code:
      - As we've seen in the EDA, the Zip code feature has a high correlation with the County of Injury and Medical Fee Region. Given That, we are dropping it

In [13]:
#remove the values 'Carrier Name' and 'Zip Code' from the list of categorical features

X_train_categorical = X_train_categorical.drop(['Carrier Name', 'Zip Code'], axis=1)
X_test_categorical = X_test_categorical.drop(['Carrier Name', 'Zip Code'], axis=1)

# Check cardinality of categorical features

X_train_categorical.nunique()




Carrier Type                          8
County of Injury                     63
District Name                         8
Industry Code                        24
Industry Code Description            20
Medical Fee Region                    5
WCIO Cause of Injury Code            77
WCIO Cause of Injury Description     74
WCIO Nature of Injury Code           56
WCIO Nature of Injury Description    56
Age Group                             4
WCIO Part Of Body Code               57
WCIO Part Of Body Description        54
Broad Body Part                       6
dtype: int64

We can also see that there are features that have a code and a given description. We will reduce the model interpretabilty but we will drop the description and keep the code. The features we are dropping are:
- Industry Code Description 
- WCIO Cause of Injury Description
- WCIO Nature of Injury Description
- WCIO Part Of Body Description

In [14]:
#remove the values 'Carrier Name' and 'Zip Code' from the list of categorical features

X_train_categorical = X_train_categorical.drop(['Industry Code Description', 'WCIO Cause of Injury Description', 'WCIO Nature of Injury Description', 'WCIO Part Of Body Description'], axis=1)
X_test_categorical = X_test_categorical.drop(['Industry Code Description', 'WCIO Cause of Injury Description', 'WCIO Nature of Injury Description', 'WCIO Part Of Body Description'], axis=1)

# Check cardinality of categorical features

X_train_categorical.nunique()

Carrier Type                   8
County of Injury              63
District Name                  8
Industry Code                 24
Medical Fee Region             5
WCIO Cause of Injury Code     77
WCIO Nature of Injury Code    56
Age Group                      4
WCIO Part Of Body Code        57
Broad Body Part                6
dtype: int64

In [15]:
ohc = OneHotEncoder(sparse_output=False, drop="first", handle_unknown="ignore")

ohc_train_feat = ohc.fit_transform(X_train_categorical)
ohc_train_feat_names = ohc.get_feature_names_out()

ohc_test_feat = ohc.transform(X_test_categorical)
ohc_test_feat_names = ohc.get_feature_names_out()

In [16]:
ohc_train_df = pd.DataFrame(ohc_train_feat, index=X_train_categorical.index, columns=ohc_train_feat_names)  # Why the index=df_ohc.index?
ohc_train_df

,Carrier Type_2A. SIF,Carrier Type_3A. SELF PUBLIC,Carrier Type_4A. SELF PRIVATE,Carrier Type_5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A),Carrier Type_5C. SPECIAL FUND - POI CARRIER WCB MENANDS,Carrier Type_5D. SPECIAL FUND - UNKNOWN,Carrier Type_UNKNOWN,County of Injury_ALLEGANY,County of Injury_BRONX,County of Injury_BROOME,...,WCIO Part Of Body Code_65.0,WCIO Part Of Body Code_66.0,WCIO Part Of Body Code_90.0,WCIO Part Of Body Code_91.0,WCIO Part Of Body Code_99.0,Broad Body Part_Lower Limbs,Broad Body Part_Multiple,Broad Body Part_Other,Broad Body Part_Trunk,Broad Body Part_Upper Limbs
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5393875,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5393091,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5393889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
5393887,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5393848,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6165331,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6165265,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6165285,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [17]:
ohc_test_df = pd.DataFrame(ohc_test_feat, index=X_test_categorical.index, columns=ohc_test_feat_names)  # Why the index=df_ohc.index?
ohc_test_df

,Carrier Type_2A. SIF,Carrier Type_3A. SELF PUBLIC,Carrier Type_4A. SELF PRIVATE,Carrier Type_5A. SPECIAL FUND - CONS. COMM. (SECT. 25-A),Carrier Type_5C. SPECIAL FUND - POI CARRIER WCB MENANDS,Carrier Type_5D. SPECIAL FUND - UNKNOWN,Carrier Type_UNKNOWN,County of Injury_ALLEGANY,County of Injury_BRONX,County of Injury_BROOME,...,WCIO Part Of Body Code_65.0,WCIO Part Of Body Code_66.0,WCIO Part Of Body Code_90.0,WCIO Part Of Body Code_91.0,WCIO Part Of Body Code_99.0,Broad Body Part_Lower Limbs,Broad Body Part_Multiple,Broad Body Part_Other,Broad Body Part_Trunk,Broad Body Part_Upper Limbs
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6166141,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6165907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6166047,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6166102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6553137,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6553119,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6553542,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


## 3.1 Data Scaling for Target

In [18]:
# Initialize the encoder
le = LabelEncoder()

# Fit and transform the target variable in the training set

y_train_encoded = le.fit_transform(y_train)

y_train_encoded_df = pd.DataFrame(y_train_encoded, columns=['Encoded Target'], index=y_train.index)

In [19]:
y_train_encoded_df

,Encoded Target
Claim Identifier,
5393875,1
5393091,3
5393889,3
5393887,1
5393848,1
...,...
6165331,1
6165265,1
6165285,1


In [20]:
X_train = pd.concat([X_train_numeric_scaled, ohc_train_df, X_train_binary], axis=1)
X_test = pd.concat([X_test_numeric_scaled, ohc_test_df, X_test_binary], axis=1)

In [21]:
X_train.to_csv('datasets/scaled_data_train_delivery1.csv')
y_train_encoded_df.to_csv('datasets/scaled_target_train_delivery1.csv')
X_test.to_csv('datasets/scaled_data_test_delivery1.csv')